In [3]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
import datetime
import os
import copy
import multiprocessing as mp
from sage.graphs.graph_coloring import vertex_coloring

writeToFile = False

multithread = True

#This is the current graph file index
currentGraphFile = 0

#This is how many G6 strings go into each file
graphFileSize = 500

#This is the current output stream
outputStream = None
    
# For a memory cost, we can use this to avoid doing the checks twice for subgraphs when we remove nodes
criticalCache = {}

# This is to test the equality between the Graph6 keys and the str() function keys
testStrCache = {}

#We can also cache the Graph6 strings
graph6Cache = {}

#This is where the g6 files will be stored
imgDirectory = "critical graphs"

#Setting this to Melvin or Christopher changes where the multiprocessing happens
method = "Melvin"

num_threads = 1

def has_valid_coloring(adjacency_dict):

    # graph6DictString = graph_to_graph6_3(adjacency_dict)
    STRdictString = str(adjacency_dict)
    
    # getter = criticalCache.get(graph6DictString, None)
    getterStr = testStrCache.get(STRdictString, None)
    
    if (getterStr != None):
            # print(criticalCache[graph6DictString] == testStrCache[STRdictString])
            return getterStr

    # Determine the maximum degree of any node in the graph
    node_degrees = [len(neighbors) for neighbors in adjacency_dict.values()]
    
    if (len(node_degrees) == 0):
        cache[dictString] = 0
        return 0
    
    max_degree = max(node_degrees)

    # Define a function that recursively tries all possible colorings
    def try_coloring(coloring, node_order):
        # If all nodes have been colored, check if the coloring is valid
        if len(coloring) == len(adjacency_dict):
            for node, neighbors in adjacency_dict.items():
                for neighbor in neighbors:
                    
                    # Normally this check isn't needed, but we need it when we remove nodes from the dictionary
                    # Of course, we need to remove nodes to check if a graph is critical
                    dictNode = coloring.get(node, None)
                    dictNeighbor = coloring.get(neighbor, None)
                    
                    if (dictNode != None and dictNeighbor != None):
                        if dictNode == dictNeighbor:
                            return 0
            
            max_color = max(coloring[node] for node in adjacency_dict.keys())

            # Return the maximum color
            return max_color

        # Otherwise, recursively try all possible color assignments for the next node
        next_node = node_order[len(coloring)]
        neighborhood = adjacency_dict[next_node]
        
        for color in range(max_degree + 1):
            if all(color != coloring.get(neighbor, None) for neighbor in neighborhood):
                coloring[next_node] = color
                
                max_color = try_coloring(coloring, node_order)
                
                if (max_color != 0):
                    # Return the maximum color
                    return max_color
        return 0

    # Generate a list of nodes ordered by degree (highest first)
    nodes = list(adjacency_dict.keys())
    node_order = sorted(nodes, key=lambda node: -len(adjacency_dict[node]))

    # Try all possible colorings starting with an empty coloring
    colors = list(range(max_degree + 1))
    chi = try_coloring({}, node_order)
    # criticalCache[graph6DictString] = chi
    testStrCache[STRdictString] = chi
    # print(cache[dictString])
    return chi + 1

def adjacencyDict(edges):
        # Build a dictionary where each edge is a key and its value is a set of neighboring edges
    adjacency_dict = {}
    for edge in edges:
        if edge[0] not in adjacency_dict:
            adjacency_dict[edge[0]] = set()
        if edge[1] not in adjacency_dict:
            adjacency_dict[edge[1]] = set()
        adjacency_dict[edge[0]].add(edge[1])
        adjacency_dict[edge[1]].add(edge[0])
    
    print(adjacency_dict)
    return adjacency_dict

def remove_node(adj_dict, node):
    # create a copy of the adjacency dictionary to modify
    new_adj_dict = copy.deepcopy(adj_dict)

    # remove the node and its edges from the dictionary
    
    #This takes care of the node
    del new_adj_dict[node]
    
    #This loops through the edges
    for neighbor in adj_dict[node]:
        
        getter = new_adj_dict.get(neighbor, None)
        if (getter != None) :
            if (node in new_adj_dict[neighbor]):
                new_adj_dict[neighbor].remove(node)
    # del new_adj_dict[node]

  # shift down the node indices higher than the removed node
    for i in range(node+1, max(new_adj_dict)+1):
        if i in new_adj_dict:
            new_adj_dict[i-1] = new_adj_dict.pop(i)
            new_adj_dict[i-1] = {j-1 if j>node else j for j in new_adj_dict[i-1]}
  

    return new_adj_dict


def graph_to_graph6_3(adj_dict):
    
    dictString = str(adj_dict)
    
    #This cache can get big, so we only do it for small graphs
    if (len(adj_dict) <= 5):
        getter = graph6Cache.get(dictString, None)
        if (getter != None):
                return getter
    
    # Create a graph object from the adjacency dictionary
    G = nx.Graph(adj_dict)

    # Obtain a Graph6 string from the graph object
    graph6 = nx.to_graph6_bytes(G).decode('ascii')

    # Remove the newline character at the end of the string
    graph6 = graph6.rstrip('\n')
    
    if (len(adj_dict) <= 5):
        graph6Cache[dictString] = graph6
    
    return graph6

def vertex_color(graph, GRAPH_K, mpVal):
    
    if mpVal.value == 0:
        return False
    
    boolean = vertex_coloring(graph, k= GRAPH_K, value_only=True)
    
    #This sets the graph as not critical if this part of the crit check fails
    if (boolean == False):
        mpVal.value = 0
        
    return boolean

#This takes a Sage graph as an argument
def isSageCriticalOnK(graph, GRAPH_K):

    global method
    
    # Set the original graph before messing with it
    original_graph = Graph(graph)

    # Get it from the graph
    chromatic_number = original_graph.chromatic_number()

    # Only if the CN is K
    if chromatic_number == GRAPH_K:

        # Get the order of the graph
        order = original_graph.order()

        # A flag to check if a graph is critical
        is_critical = True
        is_critical_value = mp.Value('i', 1)
        
        # Melvin's method multithreads here. Christopher's does not

        threadTasks = []

        taskAssign = 0

        processArray = []
        
        
        # Iterate through every vertice in graphs vertices
        for index in range(order):
            graph = Graph(original_graph)

            # Remove the vertex at index
            graph.delete_vertex(index)

            threadTasks.append(graph)

            # After deleting a vertex, can we color this graph with less colors. (Faster than re-caculating the chormatic number)
            #if vertex_color(graph, GRAPH_K - 1, is_critical_value) == False:

                # Is critical is false
                #is_critical = False

                # Not critical, lets break the loop
                #break

                
          # This actually does the real multithreading

        if method == "Melvin" or method == "Combined":
            
            for graph_ in threadTasks:
                process = mp.Process(target = vertex_color, args = (graph_, GRAPH_K - 1, is_critical_value))
                process.start()
                processArray.append(process)

            for process in processArray:
                process.join()
                process.close()
                
            if is_critical_value == 0:
                is_critical = False
            else:
                is_critical = True
                
        elif method == "Christopher":
            
            for graph_ in threadTasks:
                
                critResult = vertex_color(graph_, GRAPH_K - 1, is_critical_value)
                
                if is_critical_value == 0:
                    is_critical = False
                    break
                else:
                    is_critical = True
                
                #if (critResult == False):
                    #is_critical = False
                    #break;

        
        # Reset the graph to replace deleted vertex
        graph = Graph(original_graph)
        
        #print(graph.graph6_string())
        
        # Return is_critical
        return is_critical

    # Return not critical to k value
    return False

def removeSubstring(myStr, substring):
    output_string = ""
    str_list = myStr.split(substring)
    for element in str_list:
        output_string += element
    return output_string

def subdirectory(subdirectoryName, number):
    # Get the current directory path
    current_directory = os.path.dirname(os.path.abspath(__file__))


    # Create the subdirectory if it doesn't exist
    subdirectory_path = os.path.join(current_directory, subdirectoryName)
    os.makedirs(subdirectory_path, exist_ok=True)

    # Create the file within the subdirectory
    file_path = os.path.join(subdirectory_path, "classified graphs " + str(number) + ".txt")
    return file_path

def showGraph(dict_):
    #print(dict_)
    
    G = nx.Graph(dict_)
                        
    color_array = ["red", "blue", "green", "gold", "orange", "purple", "teal", "black", "grey", "steelblue", "magenta", "violet", "dodgerblue", "brown"]
    color_array_trunc = []
    
    for i in range(len(dict_)):
        color_array_trunc.append(color_array[i])
        
    # Create a dictionary that maps each node to its color
    # color_map = {node: color_array[coloring[node]] for node in adjacency_dict.keys()}

    # Draw the colored graph using networkx
    nx.draw(G, with_labels=True, node_color=color_array_trunc, font_color='w')
    plt.show()

def sageCritTest(graph, minK, maxK):
  
    #We're looking for 
    for i in range(minK, maxK + 1):
        
        if (isSageCriticalOnK(graph, i)):
            # print("Critical graph found!")
            return [True, "The graph is critical on: " + str(i)]
    
    return [False, "The graph is not critical in the k range of " + str(minK) + " - " + str(maxK)]


def multiFoo(graphArrays, outputLineQueue):
    
    print(len(graphArrays))
    
    
    for arr in graphArrays:
        #classifyGraph(arr, outputLineQueue)
        classifySageGraph(arr, outputLineQueue)
        
    
    outputTarget = outputLineQueue.qsize()
    print(f"{outputTarget} graphs found so far!")

def classifySageGraph(graph, outputLineQueue):
        
    global imgDirectory
    global currentGraphFile
    global outputStream

    # This array has True in index 0 if the graph is critical
    # It ha a print message in index 1
    graphResults = sageCritTest(graph, min_k, max_k)
    if (graphResults[0]):

        graph6Str = graph.graph6_string()
        graph6Str = removeSubstring(graph6Str, ">>graph6<<")

        outputLineQueue.put(graph6Str)
        #print(graph6Str)
        
        print(outputLineQueue.qsize())
            

    return graphResults[0]

def saveFiles(outputLineQueue):
    
    global writeToFile
    global currentGraphFile
    global outputStream
    
    foundGraphs = 0
    outputLength = 0
    outputTarget = outputLineQueue.qsize()
    
    
    #print(f"{outputTarget} total graphs found 2!")
    
    if (writeToFile):

        while outputLength != outputTarget:
            
            outputLine = outputLineQueue.get()
            
            if (outputLine == None):
                print("No")
                continue
            
            outputStream.write(outputLine)
            outputStream.write("\n")

            foundGraphs = foundGraphs + 1
            outputLength = outputLength + 1
            
            #print(outputLength)
            
            if (foundGraphs > 500):
                foundGraphs = 0
                
                outputStream.close()

                print("File " + str(currentGraphFile) + " Created!")
                currentGraphFile = currentGraphFile + 1

                path = os.path.join(imgDirectory, f"graph_{currentGraphFile}.g6")
                outputStream = open(path, "w")
    
        print("File " + str(currentGraphFile) + " Created!")
        outputStream.close()
    

def multithreading2(min_nodes, max_nodes, min_k, max_k):
    
    global imgDirectory
    global currentGraphFile
    global outputStream
    global num_threads
    global method
    
    if (not os.path.exists(imgDirectory)):
        os.makedirs(imgDirectory)
    
    # Create a thread pool executor with the specified number of threads
    # The thread count we use here depends on the method
    
    arrayLen = num_threads
    if (method == "Melvin"):
        arrayLen = 1
    
    #Make a 2D array for thread tasks (will become 3D later in code)
    #1st dimension stores the thread to use
    #2nd dimension stores the graph arguments that each thread is in charge of
    #3rd dimension stores the arguments for each graph
    threadTasks = []
    for i in range(arrayLen):
        threadTasks.append([])
        
    taskAssign = 0
    
    #pool = multiprocessing.Pool()
    processArray = []

    #This is the array of Graph6 outputs
    outputLineQueue = mp.Queue()

    # Submit each number to the executor for processing

    #futures = []


    path = os.path.join(imgDirectory, f"graph_{currentGraphFile}.g6")
    outputStream = open(path, "w")

    allGraphs = 0

    for num_nodes in range(min_nodes, max_nodes+1):
        nauty_generator = graphs.nauty_geng(str(num_nodes) + " -c")
        graphs_list = list(nauty_generator)

        for g in graphs_list:
            #g.show()
            networkx_graph = g.networkx_graph()

            #threadTasks[taskAssign].append([networkx_graph.edges(), networkx_graph.nodes()])
            threadTasks[taskAssign].append(g)

            taskAssign = taskAssign + 1
            allGraphs = allGraphs + 1

            if (taskAssign == arrayLen):
                taskAssign = 0



    print(f"{allGraphs} graphs searched")
    
    # This actually does the real multithreading
    #pool.map(multiFoo, threadTasks)
    
    for thread in threadTasks:
        process = mp.Process(target = multiFoo, args = (thread, outputLineQueue,))
        process.start()
        processArray.append(process)
    
    
    #This waits for the threads to finish
    #pool.close()
    #pool.join()
     
    for process in processArray:
        process.join()
        
        #print(f"{outputLineQueue.qsize()} graphs found at the end of the process!")
        process.close()
    
    print(f"{outputLineQueue.qsize()} total graphs found!")
    #results = [future.result() for future in futures]
    
    saveFiles(outputLineQueue)

    return
                
    
def useMethod(min_nodes, max_nodes, min_k, max_k, threads, method_):
    
    global num_threads
    global method
    
    num_threads = threads
    method = method_
    
    print(f"Starting using the {method_} method!")

    start_time = datetime.datetime.now()


    multithreading2(min_nodes, max_nodes, min_k, max_k)


    end_time = datetime.datetime.now()

    elapsed_time = end_time - start_time
    totalSeconds = elapsed_time.total_seconds()



    hours, remainder = divmod(int(totalSeconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    roundSeconds = float("{:.2f}".format(math.modf(totalSeconds)[0] + seconds))

    print("Elapsed time: {} hours, {} minutes, {} seconds".format(int(hours), int(minutes), roundSeconds))
    

min_nodes = int(input("Enter the min number of nodes: "))
max_nodes = int(input("Enter the max number of nodes: "))

min_k = int(input("Enter the min number of k: "))
max_k = int(input("Enter the max number of k: "))

threads = int(input("Enter the number of threads for Christopher's method: "))

useMethod(min_nodes, max_nodes, min_k, max_k, threads, "Melvin")
useMethod(min_nodes, max_nodes, min_k, max_k, threads, "Christopher")
useMethod(min_nodes, max_nodes, min_k, max_k, threads, "Combined")

Enter the min number of nodes: 6
Enter the max number of nodes: 7
Enter the min number of k: 5
Enter the max number of k: 5
Enter the number of threads for Christopher's method: 4
Starting using the Melvin method!
965 graphs searched
965
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
50 graphs found so far!
50 total graphs found!
Elapsed time: 0 hours, 0 minutes, 2.19 seconds
Starting using the Christopher method!
965 graphs searched
242
241
241
241
1
2
3
4
5
6
78

9
10
11
1312

14
15
1617

18
20
2119

22
2324

25
26
2728

2930

3131 graphs found so far!

32
33
35
3637

38
39
4034

41
4243
44

454647


47 graphs found so far!
4849

49 graphs found so far!
50
50 graphs found so far!
50 total graphs found!
Elapsed time: 0 hours, 0 minutes, 0.15 seconds
Starting using the Combined method!
965 graphs searched
242
241
241
241
13
3

4
5
6
7
8
9
111112


131514


16
1817

19
20
2122

23
24
2526

27
2

In [53]:
#Test 78662
#GCv~~{
#GEv^~{
#GEv]~{
#GEu~~{
#GEn~~{
#GUZ~~{
#GQzn^{


#Test 78664
#GEv\~{
#GEu~~{
#GEn~~{
#GQzn^{
#GUZ~~{
#GQz~~{